In [2]:
from ase import Atoms
from ase.visualize import view
import numpy as np

In [3]:
a_bcc = 3.11

In [4]:
bccTi = Atoms(cell=[a_bcc]*3, symbols='Ti2', scaled_positions=[[0, 0, 0], [0.5, 0.5, 0.5]], pbc=True)
view(bccTi, viewer='nglview')

In [16]:
positions = [np.array([i, 0, (22-i*4)%22]) for i in range(11)] + [np.array([i, 1, (11+22-i*4)%22]) for i in range(11)]
positions /= np.array([11, 2, 22])
twin_cell = Atoms(
    cell = a_bcc*np.array([5.5/(3**2+1+1)**0.5, 2/(1+1+0)**0.5, 22/(2**2+3**2+3**2)**0.5]),
    symbols = 'H22',
    scaled_positions = np.array(positions),
    pbc=True
)
view(twin_cell, viewer='nglview')

In [19]:
abeta = bccTi.cell.cellpar()[0]
bbeta = twin_cell.cell.cellpar()[1]
vbeta = twin_cell.cell.cellpar()[0]
wbeta = twin_cell.cell.cellpar()[2]
delta4 = (3*abeta**2+bbeta**2)/(4*vbeta)
abeta, bbeta, vbeta, delta4

(3.11, 4.3982041789803255, 5.157351549002646, 2.344250704092112)

For the $\delta_1$ to work, magnitude of $\delta_2$ and $\delta_3$ have to adjusted, and also final $\delta_5$ shuffle should be applied

In [53]:
# delta1 = bbeta/9
# delta1_x = delta1*(2/11)**0.5
# delta1_z = delta1*3/11**0.5

In [52]:
for zeta in np.linspace(0, 1, 22):
    ctwin = (wbeta**2+(2*zeta*delta4)**2)**0.5
    gtwin = np.arctan(zeta*delta4/wbeta)*180/np.pi
    twin = twin_cell.copy()
    pos = twin.get_positions()
    for i in range(len(pos)):
        pos[i, 0] += 0.5*2*pos[i, 2]*zeta*delta4/wbeta
        if abs(pos[i, 1]) < 0.1:
            # pos[i, 2] += zeta*wbeta/22
            # pos[i, 0] += delta1_x
            # pos[i, 2] += delta1_z
        else:
            pos[i, 2] -= zeta*wbeta/22
    twin.set_positions(pos)
    # new cell corresponding to the above displacements
    twin.set_cell([vbeta, bbeta, ctwin, 90, 90-gtwin, 90], scale_atoms=False)
    # apply additional shear, thereby doubling the shear angle
    twin.set_cell([vbeta, bbeta, ctwin, 90, 90-2*gtwin, 90], scale_atoms=True)
    twin.write(filename=f'str_z={zeta:.3f}.vasp', format='vasp')